<a href="https://colab.research.google.com/github/maheshr24/GradientBoostingRegressor-on-YT-API/blob/main/youtube_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import requests
import time

In [ ]:
API_KEY="API-KEY"
CHANNEL_ID="UCW8Ews7tdKKkBT6GdtQaXvQ"
pageToken=""

In [ ]:
url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

response = requests.get(url).json()

In [ ]:
response['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'XweHQKy3K6FKWDrDsxlyHvXa0-c',
 'id': {'kind': 'youtube#video', 'videoId': '_ZI9wfqEv_g'},
 'snippet': {'publishedAt': '2024-01-03T09:00:22Z',
  'channelId': 'UCW8Ews7tdKKkBT6GdtQaXvQ',
  'title': 'The Different Job Roles For Data Scientists In 2024',
  'description': "Dive into the dynamic world of data science! Join us as we trace the evolution from the COVID-19 boom to today's specialized ...",
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/_ZI9wfqEv_g/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/_ZI9wfqEv_g/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/_ZI9wfqEv_g/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'StrataScratch',
  'liveBroadcastContent': 'none',
  'publishTime': '2024-01-03T09:00:22Z'}}

In [ ]:
length=len(response['items'])
length

50

In [ ]:
response['items'][28]['snippet']['title']

'Python Pandas: Understanding Percentage Ranking Method'

In [ ]:
response['items'][28]['id']['videoId']

'A9QzeBFj5GY'

In [ ]:
response['items'][23]['snippet']['publishTime']

'2023-05-31T17:00:10Z'

In [ ]:
def get_video_details(video_id):

    #collecting view, like, dislike, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, comment_count

In [ ]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [ ]:
df2 = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"])

df2 = get_videos(df2)

In [ ]:
df2

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,_ZI9wfqEv_g,The Different Job Roles For Data Scientists In...,2024-01-03,1057,48,6
1,o3Yau5MJDZ4,When Does A Senior Data Scientist Actually Cod...,2023-12-06,1041,41,1
2,GSMCs3EmlmU,Short Primer on Tree-Based Machine Learning Mo...,2023-11-29,614,27,3
3,cEannyn5exA,What Visualization Frameworks Do You Need To L...,2023-11-16,1184,40,2
4,tT7STcGCvUY,Cleaning Data Like a 10x Data Scientist,2023-11-08,2035,92,2
...,...,...,...,...,...,...
177,UX4_IgagL9I,How to Use Google Colaboratory | Import a CSV ...,2020-05-02,2682,16,2
178,tDdo3FiWpgE,Interview Questions for SQL Joins and Subqueries,2020-02-01,1990,28,0
179,wW827gqxlRY,SQL Job Interview Mistakes #2,2019-09-15,1211,20,0
180,xbc2GpGUXwc,SQL Job Interview Mistakes #1,2019-08-03,8234,75,1


In [ ]:
df_final = df2.copy()  # Create a copy to avoid modifying the original DataFrame
df_final.drop('video_id', axis=1, inplace=True)
df_final

,video_title,upload_date,view_count,like_count,comment_count
0,The Different Job Roles For Data Scientists In...,2024-01-03,1057,48,6
1,When Does A Senior Data Scientist Actually Cod...,2023-12-06,1041,41,1
2,Short Primer on Tree-Based Machine Learning Mo...,2023-11-29,614,27,3
3,What Visualization Frameworks Do You Need To L...,2023-11-16,1184,40,2
4,Cleaning Data Like a 10x Data Scientist,2023-11-08,2035,92,2
...,...,...,...,...,...
177,How to Use Google Colaboratory | Import a CSV ...,2020-05-02,2682,16,2
178,Interview Questions for SQL Joins and Subqueries,2020-02-01,1990,28,0
179,SQL Job Interview Mistakes #2,2019-09-15,1211,20,0
180,SQL Job Interview Mistakes #1,2019-08-03,8234,75,1


In [ ]:
vectorizer = CountVectorizer()
X_text_features = vectorizer.fit_transform(df_final['video_title']).toarray()

In [ ]:
X = pd.concat([pd.DataFrame(X_text_features, columns=vectorizer.get_feature_names_out()), df_final[['like_count', 'comment_count']]], axis=1)
y = df_final['view_count']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(X_train, y_train)


GradientBoostingRegressor(random_state=42)

In [ ]:
y_pred=gb_model.predict(X_test)

In [ ]:

'''
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred_l = regressor.predict(X_test)
'''

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')

Mean Squared Error on Test Set: 9198300.982470687


In [ ]:
'''mse = mean_squared_error(y_test, y_pred_l)
print(f'Mean Squared Error on Test Set: {mse}')'''

Mean Squared Error on Test Set: 6208426114.676321


In [ ]:
new_data = [
    {'title': 'SQL', 'like_count': 30, 'comment_count': 9},
    {'title': 'Data', 'like_count': 12, 'comment_count': 2},
]

# Feature engineering for new data
new_X_text_features = vectorizer.transform([item['title'] for item in new_data]).toarray()
new_X = pd.concat([pd.DataFrame(new_X_text_features, columns=vectorizer.get_feature_names_out()), pd.DataFrame(new_data)[['like_count', 'comment_count']]], axis=1)

# Make predictions for new data
new_predictions = gb_model.predict(new_X)

# Print or use the predictions for new videos
for i, item in enumerate(new_data):
    print(f"Prediction for '{item['title']}': {new_predictions[i]:.2f} views")

Prediction for 'SQL': 1082.58 views
Prediction for 'Data': 635.51 views
